In [1]:
from src import utils
driver = utils.get_driver(False)

## スクレイピングの大まかな流れ
<p>以下の流れをURLのリストごとに</p>
<p>以下をページの高さが変わらなくなるまで</p>

- 5日ごとでハッシュタグ検索をする
- ツイートのリストを取得
- ランダムな時間1 ~ 1.5[s]スリープ
- 一番下までスクロール

In [3]:
from src import utils
utils.get_urls('#絵かゆ',11)

['https://twitter.com/search?q=%23%E7%B5%B5%E3%81%8B%E3%82%86since%3A2023-08-27%20until%3A2023-08-31%20min_faves%3A10&src=typed_query&f=live',
 'https://twitter.com/search?q=%23%E7%B5%B5%E3%81%8B%E3%82%86since%3A2023-09-01%20until%3A2023-09-05%20min_faves%3A10&src=typed_query&f=live',
 'https://twitter.com/search?q=%23%E7%B5%B5%E3%81%8B%E3%82%86since%3A2023-09-06%20until%3A2023-09-07%20min_faves%3A10&src=typed_query&f=live']

In [2]:
driver.get('https://twitter.com/search?q=%23%E7%B5%B5%E3%81%8B%E3%82%86since%3A2023-08-27%20until%3A2023-08-31%20min_faves%3A10&src=typed_query&f=live')

In [3]:
#Import-------------------------------------------------------------------------------
from io import StringIO, BytesIO
import os
import re
from time import sleep
import random
from urllib import request
import requests
import chromedriver_autoinstaller
from webdriver_manager.chrome import ChromeDriverManager

from selenium.common.exceptions import NoSuchElementException
from selenium import webdriver
from selenium.webdriver.chrome.options import Options as ChromeOptions
from selenium.webdriver.firefox.options import Options as FirefoxOptions
from selenium.webdriver.chrome.service import Service
import datetime
import pandas as pd
import platform
from selenium.webdriver.common.keys import Keys

from bs4 import BeautifulSoup

from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import urllib
from urllib.parse import quote

page_cards = driver.find_elements(by=By.XPATH, value='//article[@data-testid="tweet"]')

In [7]:
page_cards

[<selenium.webdriver.remote.webelement.WebElement (session="1c30d2134566d35df9b5c6af6b6fcaf1", element="6C8086E3B6B225F08390DAE02C08273C_element_126")>,
 <selenium.webdriver.remote.webelement.WebElement (session="1c30d2134566d35df9b5c6af6b6fcaf1", element="6C8086E3B6B225F08390DAE02C08273C_element_127")>,
 <selenium.webdriver.remote.webelement.WebElement (session="1c30d2134566d35df9b5c6af6b6fcaf1", element="6C8086E3B6B225F08390DAE02C08273C_element_128")>,
 <selenium.webdriver.remote.webelement.WebElement (session="1c30d2134566d35df9b5c6af6b6fcaf1", element="6C8086E3B6B225F08390DAE02C08273C_element_129")>,
 <selenium.webdriver.remote.webelement.WebElement (session="1c30d2134566d35df9b5c6af6b6fcaf1", element="6C8086E3B6B225F08390DAE02C08273C_element_130")>]

In [8]:
from src import const

In [4]:
#todo 文字や値の後処理をする
def get_data(card):
    """Extract data from tweet card"""
    image_links = []

    try:
        username = card.find_element(by=By.XPATH, value='.//span').text
    except:
        return

    try:
        handle = card.find_element(by=By.XPATH, value='.//span[contains(text(), "@")]').text
    except:
        return

    try:
        postdate = card.find_element(by=By.XPATH, value='.//time').get_attribute('datetime')
    except:
        return

    try:
        text = card.find_element(by=By.XPATH, value='.//div[2]/div[2]/div[1]').text
    except:
        text = ""

    try:
        embedded = card.find_element(by=By.XPATH, value='.//div[2]/div[2]/div[2]').text
    except:
        embedded = ""

    # text = comment + embedded

    try:
        reply_cnt = card.find_element(by=By.XPATH, value='.//div[@data-testid="reply"]').text
    except:
        reply_cnt = 0

    try:
        retweet_cnt = card.find_element(by=By.XPATH, value='.//div[@data-testid="retweet"]').text
    except:
        retweet_cnt = 0

    try:
        like_cnt = card.find_element(by=By.XPATH, value='.//div[@data-testid="like"]').text
    except:
        like_cnt = 0

    try:
        elements = card.find_elements(by=By.XPATH, value='.//div[2]/div[2]//img[contains(@src, "https://pbs.twimg.com/")]')
        for element in elements:
            image_links.append(element.get_attribute('src'))
    except:
        image_links = []

    try:
        promoted = card.find_element(by=By.XPATH, value='.//div[2]/div[2]/[last()]//span').text == "Promoted"
    except:
        promoted = False
    if promoted:
        return

    # get a string of all emojis contained in the tweet
    try:
        emoji_tags = card.find_elements(by=By.XPATH, value='.//img[contains(@src, "emoji")]')
    except:
        return
    emoji_list = []
    for tag in emoji_tags:
        try:
            filename = tag.get_attribute('src')
            emoji = chr(int(re.search(r'svg\/([a-z0-9]+)\.svg', filename).group(1), base=16))
        except AttributeError:
            continue
        if emoji:
            emoji_list.append(emoji)
    emojis = ' '.join(emoji_list)

    # tweet url
    try:
        element = card.find_element(by=By.XPATH, value='.//a[contains(@href, "/status/")]')
        tweet_url = element.get_attribute('href')
    except:
        return

    tweet = (
        username, handle, postdate, text, embedded, emojis, reply_cnt, retweet_cnt, like_cnt, image_links, tweet_url)
    return tweet

In [10]:
tweets = []
for card in page_cards:
    tweet = get_data(card)
    tweets.append(tweet)

for t in tweets:
    print(t)

('保村 成(ほむらみのり)', '@izumochi01', '2023-08-30T23:22:37.000Z', '保村 成(ほむらみのり)\n@izumochi01\n·\n8月31日', '#絵かゆ\nダイヤを取り立てるねこねこふぁいにゃんすの猫又社長の図\n「簡単に手にしたダイヤは簡単に使うんだよねぇ…」\n\nリプ欄にタイムラプスとかいろいろ置いとく', '', '3', '172', '751', ['https://pbs.twimg.com/media/F40NIBybEAAa4Fe?format=webp&name=360x360', 'https://pbs.twimg.com/media/F40NIB0akAAoa9c?format=webp&name=small'], 'https://twitter.com/izumochi01/status/1697027071288213635')
('IKAYAKI\u3000呑み過ぎシリーズ', '@ikayaki_4', '2023-08-30T23:04:14.000Z', 'IKAYAKI\u3000呑み過ぎシリーズ\n@ikayaki_4\n·\n8月31日', '#絵かゆ #シオンの書物  #gawrt\nNEKO3', '🦑 🔥 📦 💨 ✍ 🦑 ♥', '10', '1,117', '7,134', ['https://pbs.twimg.com/media/F40I6N6a8AAvKNh?format=webp&name=small'], 'https://twitter.com/ikayaki_4/status/1697022445235089651')
None
('Juunto', '@JuuntoArt', '2023-08-30T19:16:38.000Z', 'Juunto\n@JuuntoArt\n·\n8月31日', '#絵かゆ Nekomata Okayu Hololive Feet GAMERS.', '', '', '4', '18', ['https://pbs.twimg.com/media/F4zU0p2aMAQUDCg?format=webp&name=small'], 'https://twitter.com/JuuntoArt/status/169

In [5]:
def get_page_height(driver):
    page_height = driver.execute_script("return Math.max( document.body.scrollHeight, document.body.offsetHeight, document.documentElement.clientHeight, document.documentElement.scrollHeight, document.documentElement.offsetHeight );")
    return page_height

In [6]:
before_height = 0
after_height = get_page_height(driver)
tweets = []
#一番下に行くまで
while(before_height != after_height):
    page_cards = driver.find_elements(by=By.XPATH, value='//article[@data-testid="tweet"]')
    for card in page_cards:
        tweet = get_data(card)
        tweets.append(tweet)
    sleep(random.uniform(1, 2))
    before_height = after_height
    driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
    after_height = get_page_height(driver)
    print(f"\r tweets -> {len(tweets)} ",end="")

 tweets -> 38 

In [7]:
tweets

[('保村 成(ほむらみのり)',
  '@izumochi01',
  '2023-08-30T23:22:37.000Z',
  '保村 成(ほむらみのり)\n@izumochi01\n·\n8月31日',
  '#絵かゆ\nダイヤを取り立てるねこねこふぁいにゃんすの猫又社長の図\n「簡単に手にしたダイヤは簡単に使うんだよねぇ…」\n\nリプ欄にタイムラプスとかいろいろ置いとく',
  '',
  '3',
  '172',
  '751',
  ['https://pbs.twimg.com/media/F40NIBybEAAa4Fe?format=jpg&name=360x360',
   'https://pbs.twimg.com/media/F40NIB0akAAoa9c?format=jpg&name=small'],
  'https://twitter.com/izumochi01/status/1697027071288213635'),
 ('IKAYAKI\u3000呑み過ぎシリーズ',
  '@ikayaki_4',
  '2023-08-30T23:04:14.000Z',
  'IKAYAKI\u3000呑み過ぎシリーズ\n@ikayaki_4\n·\n8月31日',
  '#絵かゆ #シオンの書物  #gawrt\nNEKO3',
  '🦑 🔥 📦 💨 ✍ 🦑 ♥',
  '10',
  '1,117',
  '7,134',
  ['https://pbs.twimg.com/media/F40I6N6a8AAvKNh?format=jpg&name=small'],
  'https://twitter.com/ikayaki_4/status/1697022445235089651'),
 None,
 ('Juunto',
  '@JuuntoArt',
  '2023-08-30T19:16:38.000Z',
  'Juunto\n@JuuntoArt\n·\n8月31日',
  '#絵かゆ Nekomata Okayu Hololive Feet GAMERS.',
  '',
  '',
  '4',
  '18',
  ['https://pbs.twimg.com/media/F4zU0p2aMAQUDCg?fo